# Cat vs Dog Classifier

This is a solution for the Dog vs cat kaggle competition
https://www.kaggle.com/c/dogs-vs-cats/

Using Convolutional Neural Network

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import pandas as pd
from sklearn.model_selection import train_test_split 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import random

#TRAIN_DIR = '/home/phrc/Documents/ml3 assigment/Project3_files/data/train'
TRAIN_DIR = '/home/phrc/Documents/ml3 assigment/images/train'

TEST_DIR = '/home/phrc/Documents/ml3 assigment/Project3_files/data/test'
IMG_SIZE = 96

import tensorflow as tf
tf.reset_default_graph()

gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.8)
session = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

In [2]:
def labelImage(img):
    label = img.split('.')[0]
    if label == 'cat': 
        return 1
    elif label == 'dog': 
        return 0
    else:
        return -1
    
#def createData(directory, color=cv2.IMREAD_GRAYSCALE):
def createData(directory, color=cv2.COLOR_RGBA2RGB):
    X = []
    for img in os.listdir(directory):
        label = labelImage(img)
        img = cv2.imread(os.path.join(directory,img), color)
        img  = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        X.append([img, label])
    return X

In [3]:
training_data = createData(TRAIN_DIR)

random.shuffle(training_data)

x_data = []
y_data = []

for features,label in training_data:
    x_data.append(features)
    y_data.append(label)


x_data = np.array(x_data).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
x_data = x_data/255.0

print(len(x_data))

25000


In [4]:
def createModel(convSizeValues):

    model = Sequential()

    model.add(Conv2D(convSizeValues[0], (3, 3), input_shape=x_data.shape[1:]))
    model.add(Activation('relu'))
    
    for i in range(1, len(convSizeValues)):
        model.add(Conv2D(convSizeValues[i], (3, 3)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

    model.add(Dense(64))

    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    tensorboard = TensorBoard(log_dir="log/{0} - {1}".format(len(convSizeValues), ','.join([str(i) for i in convSizeValues])), histogram_freq=1, write_graph=True, write_images=True)

    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'],
                  )

    model.fit(x_data, y_data,
              batch_size=32,
              epochs=10,
              validation_split=0.3,
              callbacks=[tensorboard])


In [5]:
#model1 = createModel([32, 32, 32])
#model2 = createModel([64, 64, 64])
model3 = createModel([128, 128, 128])
#model4 = createModel([32, 64, 128])

Instructions for updating:
Colocations handled automatically by placer.
Train on 17500 samples, validate on 7500 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
17500/17500 [==============================] - 591s 34ms/sample - loss: 0.6853 - acc: 0.5877 - val_loss: 0.6322 - val_acc: 0.6484
Epoch 2/10
17500/17500 [==============================] - 639s 37ms/sample - loss: 0.6029 - acc: 0.6829 - val_loss: 0.5868 - val_acc: 0.6941
Epoch 3/10
17500/17500 [==============================] - 765s 44ms/sample - loss: 0.5553 - acc: 0.7232 - val_loss: 0.5201 - val_acc: 0.7496
Epoch 4/10
17500/17500 [==============================] - 671s 38ms/sample - loss: 0.4750 - acc: 0.7779 - val_loss: 0.4865 - val_acc: 0.7651
Epoch 5/10
17500/17500 [==============================] - 785s 45ms/sample - loss: 0.4149 - acc: 0.8101 - val_loss: 0.4544 - val_acc: 0.7899
Epoch 6/10
17500/17500 [==============================] - 824s 47ms/sample - loss: 0.3643 - acc: 0.8377 - val_loss: 0.4648 - v

In [3]:
%load_ext tensorboard.notebook
%tensorboard --logdir log

The tensorboard.notebook extension is already loaded. To reload it, use:
  %reload_ext tensorboard.notebook


Reusing TensorBoard on port 6006 (pid 11890), started 0:00:38 ago. (Use '!kill 11890' to kill it.)